##Task 2: Data Preparation

Preparing train and test set


 for all models after the following

1.   Creating category vector for each business id from frequent 100 categories across
2.   Adding users features like the user fans, review counts, ratings, and useful votes









Imports go here

In [0]:
from keras.preprocessing import sequence
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.datasets import imdb
import json
from pandas.io.json import json_normalize
from collections import Counter
import copy
import pandas as pd
import spacy
import numpy as np
import seaborn as sns
import nltk
import tensorflow as tf
from sklearn.metrics import roc_auc_score

#nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nlp = spacy.load('en')
from sklearn.model_selection import train_test_split
import pickle

from keras import backend as K
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from gensim.models import KeyedVectors

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  

Mounted at /content/drive


Reading review data where reviews are cleaned and filtered for review date < 2017 and length of review) >= 5

In [0]:
##reviews_data
reviews_final_df = pd.read_csv('/content/drive/My Drive/Task2_Search/reviews_final_df_fulldata.csv')
business_in_review_data = reviews_final_df['business_id'].values

##business_data
business_df = pd.read_csv('/content/drive/My Drive/Task2_Search/BusinessOrigFile.csv') 

In [0]:
business_df_filtered = business_df[business_df['business_id'].isin(business_in_review_data)]
business_df_filtered = business_df_filtered.reset_index(drop=True)

Creating Category Vector

In [0]:
#Category Vector
temp_array = business_df_filtered['categories'].values
categories = []
categories_array = []
for i in temp_array:
  categories_array.append(i.split(','))
  for j in i.split(','):
    categories.append(j)

distinct_categories = set(categories)

cleaned_categories = [w.strip() for w in categories]

categories_dict = Counter(cleaned_categories)

top100_categories = sorted(categories_dict, key=categories_dict.get, reverse=True)
top100_categories = top100_categories[0:100]


In [0]:
def create_vector(list_entry):
  vector = [0] * len(top100_categories)
  for i in range(len(top100_categories)):
    if top100_categories[i] in list_entry:
      vector[i] = 1
  return vector

In [0]:
vector_array = []
vector_df_array = []
for i in categories_array:
  result = create_vector(i)
  vector_array.append(np.array(result))
  vector_df_array.append(result)
  

In [0]:
business_df_filtered['category_vector'] = vector_df_array
np_top100_categories = np.array(top100_categories)

In [0]:
data_frame_dict = {}
for i in range(len(np_top100_categories)):
  data_frame_dict[np_top100_categories[i]] = []
  for j in range(len(vector_array)):
    data_frame_dict[np_top100_categories[i]].append(vector_array[j][i])

In [0]:
category_table = pd.DataFrame.from_dict(data_frame_dict)
business_df = pd.concat([business_df_filtered,category_table],axis=1)

In [0]:
business_df = business_df.drop(['address','attributes','categories','hours','latitude','longitude','name','city','state'],axis = 1)
merge_df = reviews_final_df.merge(business_df, on = 'business_id')
merge_df.head(2)

,business_id,user_id,review_id,cleaned_text_final,stars_x,useful,class_useful,year_date,is_open,postal_code,review_count,stars_y,category_vector,Restaurants,Food,American (Traditional),Nightlife,Bars,Sandwiches,Fast Food,American (New),Pizza,Mexican,Burgers,Breakfast & Brunch,Italian,Chinese,Coffee & Tea,Salad,Event Planning & Services,Chicken Wings,Seafood,Sports Bars,Delis,Cafes,Japanese,Caterers,Steakhouses,Sushi Bars,Bakeries,...,Venues & Event Spaces,Latin American,Hotels & Travel,Shopping,Food Trucks,Korean,Music Venues,Gastropubs,Local Flavor,Chicken Shop,French,Hotels,Tapas/Small Plates,Beer Bar,Hawaiian,Active Life,Soul Food,Dive Bars,Noodles,Donuts,Food Stands,Breweries,Cajun/Creole,Casinos,Cheesesteaks,Caribbean,Tapas Bars,Karaoke,Street Vendors,Pakistani,Halal,Irish,Tacos,Dance Clubs,Fish & Chips,Filipino,Party & Event Planning,Spanish,Meat Shops,Convenience Stores
0,5ma1OUQWy_Ds80xUiP7JPQ,pEKFes5cKIO4kl1tqqUy-g,eRZpzG1i6rPzk57KLtFtLg,reason I giving star Katsu chicken good flavor...,4.0,0.0,0,2016,1,89146,490,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5ma1OUQWy_Ds80xUiP7JPQ,SgcNTpx_ASfvyQHmLU443A,Tm9vGwhwxWQOJdg3I4j2vQ,I glad location opened strip hassle I hunt tru...,5.0,0.0,0,2016,1,89146,490,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
merge_df = merge_df.rename(columns={"stars_x": "user_rating_of_business", "stars_y": "business_rating"})


In [0]:
merge_df.head()

,business_id,user_id,review_id,cleaned_text_final,user_rating_of_business,useful,class_useful,year_date,is_open,postal_code,review_count,business_rating,category_vector,Restaurants,Food,American (Traditional),Nightlife,Bars,Sandwiches,Fast Food,American (New),Pizza,Mexican,Burgers,Breakfast & Brunch,Italian,Chinese,Coffee & Tea,Salad,Event Planning & Services,Chicken Wings,Seafood,Sports Bars,Delis,Cafes,Japanese,Caterers,Steakhouses,Sushi Bars,Bakeries,...,Venues & Event Spaces,Latin American,Hotels & Travel,Shopping,Food Trucks,Korean,Music Venues,Gastropubs,Local Flavor,Chicken Shop,French,Hotels,Tapas/Small Plates,Beer Bar,Hawaiian,Active Life,Soul Food,Dive Bars,Noodles,Donuts,Food Stands,Breweries,Cajun/Creole,Casinos,Cheesesteaks,Caribbean,Tapas Bars,Karaoke,Street Vendors,Pakistani,Halal,Irish,Tacos,Dance Clubs,Fish & Chips,Filipino,Party & Event Planning,Spanish,Meat Shops,Convenience Stores
0,5ma1OUQWy_Ds80xUiP7JPQ,pEKFes5cKIO4kl1tqqUy-g,eRZpzG1i6rPzk57KLtFtLg,reason I giving star Katsu chicken good flavor...,4.0,0.0,0,2016,1,89146,490,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5ma1OUQWy_Ds80xUiP7JPQ,SgcNTpx_ASfvyQHmLU443A,Tm9vGwhwxWQOJdg3I4j2vQ,I glad location opened strip hassle I hunt tru...,5.0,0.0,0,2016,1,89146,490,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5ma1OUQWy_Ds80xUiP7JPQ,Z1elyh9rDtx4S22uIo75-w,i5AJxathWR57MAjktRpNGw,First time I tried I good taste new like I sai...,5.0,0.0,0,2016,1,89146,490,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5ma1OUQWy_Ds80xUiP7JPQ,1z0yKHr5KBlfJqm6O11V4g,NidG6CAwEBapRdfrpP9FvQ,Not sure took long try place I days row So del...,5.0,0.0,0,2016,1,89146,490,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5ma1OUQWy_Ds80xUiP7JPQ,eiSIwHugjw9GlbqCgbER8A,zxOscKUrPfa7G6ll1ea2xA,When I Vegas I inexplicable craving good burge...,3.0,0.0,0,2016,1,89146,490,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Getting User Features

In [0]:
## Reading Users data:

'''
import json
import pandas as pd
user_data = []

with open('user.json') as f:
    for line in f:
        user_data.append(json.loads(line))

user_data = pd.DataFrame.from_dict(user_data)
user_data.head()

user_data.to_csv("userOrigFile.csv",index = False)
'''

users_data = pd.read_csv('/content/drive/My Drive/Task2_Search/userOrigFile.csv')
users_data["useful_votes_user"]=users_data["useful"]
user_data_df = users_data.loc[:, ['user_id','average_stars', 'review_count', 'useful_votes_user','fans']]
user_data_df.head()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,average_stars,review_count,useful_votes_user,fans
0,l6BmjZMeQD3rDxWUbiAiow,4.03,95,84,5
1,4XChL029mKr5hydo79Ljxg,3.63,33,48,4
2,bc8C_eETBWL0olvFSJJd0w,3.71,16,28,0
3,dD0gZpBctWGdWo9WlGuhlA,4.85,17,30,5
4,MM4RJAeH6yuaN8oZDSt0RA,4.08,361,1114,39


In [0]:
result_df = pd.merge(merge_df,
                 user_data_df,
                 on='user_id', 
                 how='left')


In [0]:
reviews_data_ML = copy.deepcopy(result_df)
reviews_data_to_train = reviews_data_ML.drop(['year_date','business_id','user_id','useful','postal_code','is_open'],axis=1)
reviews_data_to_train = reviews_data_to_train.rename(columns={"stars_x": "review_rating", "stars_y": "business_rating", "review_count_x":"business_review_count","review_count_y":"user_review_count"})
reviews_data_to_train.head(2)


,review_id,cleaned_text_final,user_rating_of_business,class_useful,business_review_count,business_rating,category_vector,Restaurants,Food,American (Traditional),Nightlife,Bars,Sandwiches,Fast Food,American (New),Pizza,Mexican,Burgers,Breakfast & Brunch,Italian,Chinese,Coffee & Tea,Salad,Event Planning & Services,Chicken Wings,Seafood,Sports Bars,Delis,Cafes,Japanese,Caterers,Steakhouses,Sushi Bars,Bakeries,Asian Fusion,Barbeque,Specialty Food,Desserts,Diners,Beer,...,Food Trucks,Korean,Music Venues,Gastropubs,Local Flavor,Chicken Shop,French,Hotels,Tapas/Small Plates,Beer Bar,Hawaiian,Active Life,Soul Food,Dive Bars,Noodles,Donuts,Food Stands,Breweries,Cajun/Creole,Casinos,Cheesesteaks,Caribbean,Tapas Bars,Karaoke,Street Vendors,Pakistani,Halal,Irish,Tacos,Dance Clubs,Fish & Chips,Filipino,Party & Event Planning,Spanish,Meat Shops,Convenience Stores,average_stars,user_review_count,useful_votes_user,fans
0,eRZpzG1i6rPzk57KLtFtLg,reason I giving star Katsu chicken good flavor...,4.0,0,490,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.73,10,11,0
1,Tm9vGwhwxWQOJdg3I4j2vQ,I glad location opened strip hassle I hunt tru...,5.0,0,490,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.62,101,62,0


Train Test Split

In [0]:
reviews_data_train, reviews_data_test = train_test_split(reviews_data_to_train, test_size=0.33, random_state=42)
reviews_data_train.reset_index().drop(['index'],axis=1).to_csv('/content/drive/My Drive/Task2_Search/TrainTestFiles/reviews_data_train.csv',index=False)
reviews_data_test.reset_index().drop(['index'],axis=1).to_csv('/content/drive/My Drive/Task2_Search/TrainTestFiles/reviews_data_test.csv',index=False)



In [0]:
len(reviews_data_test),len(reviews_data_train)

(131885, 267764)